In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self, sample_input, num_layers, neurons_per_layer):
        self.layers = self.create_layers(sample_input, num_layers, neurons_per_layer)
        self.weights = self.create_weights()
        self.biases = [0 for i in range(num_layers+1)]
        
    @classmethod
    def create_layers(self, sample_input, num_layers, neurons_per_layer):
        """ function to create hidden layers, input/output layers """
        input_layer = [0 for i in sample_input]
        output_layer = [0 for i in range(3)]
        layers = [0] * (num_layers+1)
        layers[0] = np.reshape(input_layer,(-1,1))
        # for num of layers
        for i in range(num_layers):
            # create a empty list for neurons
            current_layer = [[0] for j in  range(neurons_per_layer[i])]
            layers[i+1] = np.reshape(current_layer, (-1,1))
        layers.append(np.reshape(output_layer, (-1,1)))
        return layers

    def create_weights(self):
        weights = []
        layers = self.layers
        for i in range(len(layers)-1):
            # get the length of the current layer
            current_layer = len(layers[i])
            # get the length of the next layer
            next_layer = len(layers[i+1])
            # create a matrix and append to list using the lengths of the layers 
            weights.append(np.random.rand(next_layer, current_layer))
        return weights 
        
        
    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))
    
    def feed_forward(self, X):
        # select the layers
        layers = self.layers
        # set the input as the input layer
        layers[0] = np.reshape(X, (-1,1))
        # feed the input forward
        for i in range(len(layers)-1):
            activations = self.sigmoid(np.dot(self.weights[i], layers[i]) + self.biases[i])
            layers[i+1] = activations.copy()
        return (layers[-1], max(layers[-1]))
    
    
    def back_propagation(self, output, y):
        layers = self.layers
        weights = self.weights
        biases = self.biases
    
        for i in range(1, len(layers)):
            # select the layer (this will be -1 and then -2 so)
            # so it will be in reverse 
            activations = layers[-i]
            # select the layer before the current layer
            prev_activations = layers[-(i+1)]
            # select the weights
            weight = weights[-i]
            # select the bias
            bias = biases[-i]
            # this is the cost function
            cost_function = 2 * (activations-y) if -i == -1 else 2 * (activations-prev_activations)
#             cost_function = 2 * (activations-y)
            z = np.dot(weight, prev_activations) + bias
            dc_dw = prev_activations.T * (self.sigmoid(z)*(1-self.sigmoid(z)))  * cost_function
            dc_db =  (self.sigmoid(z)*(1-self.sigmoid(z)))  * cost_function
            biases[-i] = dc_db
            weights[-i] = dc_dw
        return 0

In [4]:
X = [X_train.iloc[1]['sepal length (cm)'], X_train.iloc[1]['sepal width (cm)'], X_train.iloc[1]['petal length (cm)'], X_train.iloc[1]['petal length (cm)']]

In [5]:
new_class = Network(X,2, [4,3])

In [6]:
last_layer, predicted = new_class.feed_forward(X)

In [7]:
y = y_train.iloc[1]['class']
y

'Iris-setosa'

In [8]:
new_class.back_propagation(last_layer, 1)

ValueError: operands could not be broadcast together with shapes (3,1) (4,1) 

In [ ]:
sample = [X_train.iloc[2]['sepal length (cm)'], X_train.iloc[2]['sepal width (cm)'], X_train.iloc[2]['petal length (cm)'], X_train.iloc[2]['petal length (cm)']]

In [ ]:
print(new_class.feed_forward(sample))